In [4]:
# imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.table import SimpleTable

## Reading Data

Reading in the data from Canada and adding a constant.

In [45]:
df = pd.read_csv('Canada.csv').drop(['COUNTRY', 'PERWT'], axis=1)
df['constant'] = 1
df['female'] = (df['SEX'] == 'Female').astype(float)
df.drop('SEX', axis=1, inplace=True)
print(df.shape)
df.head()

(724237, 10)


,AGE,MARST,NATIVITY,EDATTAIN,EMPSTAT,OCCISCO,INDGEN,INCTOT,constant,female
0,72,Widowed,Native-born,Less than primary completed,Inactive,NIU (not in universe),NIU (not in universe),9.961756,1,0.0
1,32,Single/never married,Native-born,Secondary completed,Employed,Crafts and related trades workers,Other services,10.694215,1,0.0
2,40,Married/in union,Native-born,Secondary completed,Employed,Clerks,Public administration and defense,10.956056,1,0.0
3,37,Married/in union,Native-born,Secondary completed,Employed,Technicians and associate professionals,Wholesale and retail trade,10.515967,1,1.0
4,51,Married/in union,Native-born,Secondary completed,Employed,Technicians and associate professionals,"Transportation, storage and communications",10.138560,1,1.0


In [46]:
# adding dummy variables for categorical vars, and dropping the first one to remove colinearity
dummies = pd.get_dummies(df[['MARST', 'NATIVITY', 'EDATTAIN', 'EMPSTAT', 'OCCISCO', 'INDGEN']], drop_first=True)
df = pd.concat([df[['INCTOT']], dummies, df[['female', 'AGE', 'constant']]], axis=1)

In [108]:
# exported so that we could compare to Stata
#df.to_csv('modified_canada.csv', index=False)

### Blinder-Oaxaca Prep

We need to split our data into two sets, one for females and one for males.

In [24]:
male = df[df['female']==0].reset_index(drop=True).drop('female', axis=1)
female = df[df['female']==1].reset_index(drop=True).drop('female', axis=1)

Xm = male.drop('INCTOT', axis=1)
Xf = female.drop('INCTOT', axis=1)
Ym = male['INCTOT']
Yf = female['INCTOT']

We then use that to make two regression models, using `statsmodels`'s OLS.

In [47]:
female_model = sm.OLS(Yf, Xf).fit()
male_model = sm.OLS(Ym, Xm).fit()

From these models, we are able to access $X_f$, $X_m$, $\beta_f$, and $\beta_m$, which are necessary to solve our equations. 

In [49]:
Bf = female_model.params
Bm = male_model.params

## Decomposition

Our goal is to attribute the gap between two groups to certain differences. The wage gap between males and females is defined as 

$$Gap = E[Wage_m] - E[Wage_f]$$

The gap is computed below (remember that we are dealing with log wages).

In [66]:
male_mean = np.mean(Ym)
female_mean = np.mean(Yf)

gap = male_mean - female_mean
gap

0.3037566274879566

### Three-fold Decomposition

The three-fold decomposition is split into three parts, Endowments, Coefficients, and Interactions.

$$Gap = Endowments + Coefficients + Interactions$$

Those are calculated individually below.

$$Endowments = (E[X_m]-E[X_f])'\beta_f$$

In [93]:
endowments = np.transpose(np.mean(Xm) - np.mean(Xf)) @ np.array(Bf)
endowments

0.075722997634111083

$$Coefficients = E[X_f]'(\beta_m-\beta_f)$$

In [92]:
coefficients = np.transpose(np.mean(Xf)) @ np.array(Bm - Bf)
coefficients

0.22345359683362187

$$Interactions = (E[X_m]-E[X_f])'(\beta_m-\beta_f)$$

In [69]:
interaction = np.transpose(np.mean(Xm) - np.mean(Xf)) @ np.array(Bm - Bf)
interaction

0.0045800330209516734

We are able to individually solve for the contribution from the coefficients. We calculate the contribution that comes from each variable for endowments in the cell below.

In [72]:
# variable contributions to endowments
(np.mean(Xm) - np.mean(Xf)) * Bf

MARST_Separated/divorced/spouse absent               -0.002493
MARST_Single/never married                           -0.011740
MARST_Widowed                                        -0.019391
NATIVITY_Native-born                                  0.001543
NATIVITY_Unknown/missing                              0.000001
EDATTAIN_Secondary completed                          0.003757
EDATTAIN_University completed                        -0.010440
EMPSTAT_Inactive                                      0.047626
EMPSTAT_Unemployed                                   -0.005272
OCCISCO_Crafts and related trades workers             0.018959
OCCISCO_Elementary occupations                       -0.005013
OCCISCO_Legislators, senior officials and managers    0.023513
OCCISCO_NIU (not in universe)                         0.006788
OCCISCO_Plant and machine operators and assemblers   -0.004656
OCCISCO_Professionals                                -0.009733
OCCISCO_Service workers and shop and market sales     0

### Two-fold Decomposition

$$Gap = Explained + Unexplained$$

First we have to calculate our pooled model, from which we will get our $\beta^*$.

In [87]:
pooled_model = sm.OLS(df['INCTOT'], df.drop('INCTOT', axis=1)).fit()
B_star = pooled_model.params.drop('female')

$$E = (E[X_m]-E[X_f])\beta^* $$

In [100]:
explained = np.transpose(np.mean(Xm) - np.mean(Xf)) @ np.array(B_star)
explained

0.078886205651931746

$$U = E[X_m]'(\beta_m-\beta^*) + E[X_f]'(\beta^*-\beta_f) $$

In [99]:
unexplained = np.transpose(np.mean(Xm)) @ np.array(Bm-B_star) + np.transpose(np.mean(Xf)) @ np.array(B_star-Bf)
unexplained

0.22487042183675293

## Summary Tables

We use the `statsmodels` implementation of `SimpleTable` to build our summary tables. We compare our results to Stata's outputs using the same data.

In [103]:
mydata = [[group1], [group2], [difference], [endowments], [coefficients], [interaction]]
myheaders = ["Coef."]#, "Std. Err.", "z"
mystubs = ["group_1", "group_2", "difference", "endowments", "coefficients", "interaction"]
tbl = SimpleTable(mydata, myheaders, mystubs, title="Blinder-Oaxaca Decomposition: Three-Fold")
tbl

,Coef.
group_1,10.234547702209856
group_2,9.930791074721899
difference,0.3037566274879566
endowments,0.0757229976341
coefficients,0.223453596834
interaction,0.00458003302095


![](images/three-fold.png)

In [102]:
mydata = [[group1], [group2], [difference], [explained], [unexplained]]
myheaders = ["Coef."]#, "Std. Err.", "z"
mystubs = ["group_1", "group_2", "difference", "explained", "unexplained"]
tbl = SimpleTable(mydata, myheaders, mystubs, title="Blinder-Oaxaca Decomposition: Pooled")
tbl

,Coef.
group_1,10.234547702209856
group_2,9.930791074721899
difference,0.3037566274879566
explained,0.0788862056519
unexplained,0.224870421837


![](images/pooled.png)

In [44]:
mydata = [[co] for co in cfs]  # data MUST be 2-dimensional
myheaders = [ "Coef."]#, "Std. Err.", "z"
mystubs = list(cfs.index)
tbl = SimpleTable(mydata, myheaders, mystubs, title="Blinder-Oaxaca Decomposition - $C$")
tbl

,Coef.
MARST_Separated/divorced/spouse absent,-0.0271724652886
MARST_Single/never married,-0.0585922945188
MARST_Widowed,-0.0235621121724
NATIVITY_Native-born,0.101755124514
NATIVITY_Unknown/missing,-1.44134985957e-06
EDATTAIN_Secondary completed,0.031518105719
EDATTAIN_University completed,0.00781105200052
EMPSTAT_Inactive,0.017192818169
EMPSTAT_Unemployed,-0.00269069173622
OCCISCO_Crafts and related trades workers,0.000563035836238


In [73]:
# coefficients
cfs = np.mean(Xf) * (male_model.params - female_model.params)

In [57]:
group1 = np.mean(Ym)
group1

10.234547702209856

In [58]:
group2 = np.mean(Yf)
group2

9.930791074721899

In [59]:
difference = np.mean(Ym) - np.mean(Yf)
difference

0.3037566274879566

In [35]:
male_model.params.var()

2.1010798689470844

In [36]:
np.transpose(np.array(male_model.params)) @ np.array(Xm.var() + Xf.var())

16.251015844938422

In [37]:
np.dot(np.transpose(np.array(male_model.params)),np.array(Xm.var() + Xf.var()))

16.251015844938422

In [38]:
np.transpose(np.array(male_model.params)) @ np.array(Xm.var() + Xf.var()) @ np.array(male_model.params)

ValueError: Scalar operands are not allowed, use '*' instead

In [39]:
np.transpose(np.array(male_model.params)) * (np.mean(Xm).var() - np.mean(Xf).var()) @ np.array(male_model.params)

-204.21219897599974

In [40]:
(np.transpose(np.array(np.mean(Xm) - np.mean(Xf))) * male_model.params.var()) @ np.array(np.mean(Xm) - np.mean(Xf))

2.1797058594948919

In [41]:
# interactions
(np.mean(Xm) - np.mean(Xf)) * (male_model.params - female_model.params)

MARST_Separated/divorced/spouse absent                8.533520e-03
MARST_Single/never married                           -1.248964e-02
MARST_Widowed                                         1.743828e-02
NATIVITY_Native-born                                  1.191870e-03
NATIVITY_Unknown/missing                             -2.501063e-07
EDATTAIN_Secondary completed                          1.030447e-03
EDATTAIN_University completed                        -6.232971e-04
EMPSTAT_Inactive                                     -4.308824e-03
EMPSTAT_Unemployed                                   -6.422489e-04
OCCISCO_Crafts and related trades workers             9.066435e-03
OCCISCO_Elementary occupations                        2.620443e-03
OCCISCO_Legislators, senior officials and managers   -2.931533e-03
OCCISCO_NIU (not in universe)                        -1.940097e-03
OCCISCO_Plant and machine operators and assemblers    6.519700e-03
OCCISCO_Professionals                                 8.930786

oaxaca inctot mar* nativity* edattain* emp* occisco* indgen* age, by(female) pooled relax